## Measuring VNC length

The VNC length is measured using a centerline estimation.
The steps are as follows:

1. The image is binarized, so we can apply a distance transform.
The distance transform is very succeptible to holes and concavities, so we also remove holes from the binary image and apply a binary opening to smooth edges.
2. We apply a 'chessboard' distance transform (minimum difference between two points along any coordinate dimension) to the binary image, which naturally results in peaks over the centerline of the binary image.
Given the embryo morphology, we expect several center points along the anterior-posterior axis and eventually another set of points towards the brain lobes.
3. To separate these points, we fit a RANSAC model that, given the majority of the points are along the AP axis, fits a line along the VNC centerline.
4. From this line intersection to the binary image, we are able to measure the VNC length.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
from pathlib import Path

from tifffile import imread
import matplotlib.pyplot as plt

from pasnascope import roi, vnc_length, find_hatching

experiment_name = '20240307'
root_dir = Path(os.getcwd()).parent
project_dir = os.path.join(root_dir, 'data', experiment_name)

In [ ]:
def get_length_centerline(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, without masking.'''
    img = imread(file_path, key=range(0,hp, interval))
    return vnc_length.measure_VNC_centerline(img)*pixel_width

interval = 20
img_dir = os.path.join(project_dir, 'embs')
annotated_dir = os.path.join(project_dir, 'annotated')
embs = [f.split('.')[0] for f in os.listdir(annotated_dir)][:2]
print(embs)

lengths = []
ann_lengths = []

for emb in embs:
    img_path = os.path.join(img_dir, f"{emb}.tif")
    hp = find_hatching.find_hatching_point(img_path)
    # adjust hatching point to make it comparable with annotated data
    hp -= hp % interval

    vnc_len = get_length_centerline(img_path, hp, interval=interval)
    lengths.append(vnc_len)

    csv_path = os.path.join(annotated_dir, f"{emb}.csv")
    csv_end = hp//interval
    manual = vnc_length.get_length_from_csv(csv_path, csv_end)
    ann_lengths.append(manual)

fig, ax = plt.subplots(len(embs))
ax = ax.ravel()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle('Centerline estimation')
for i, (l, al) in enumerate(zip(lengths, ann_lengths)):
    x = list(range(0, len(l)*interval, interval))
    ax[i].plot(x, l, color='r', label='calculated')
    ax[i].plot(x, al, color='g', label='annotated')
ax[0].legend()

plt.tight_layout()

Visualize how much variability in the centerline estimation we have, since RANSAC in inherently non-deterministic.

In [ ]:

def get_length_centerline(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, without masking.'''
    img = imread(file_path, key=range(0,hp, interval))
    return vnc_length.measure_VNC_centerline(img)*pixel_width

interval = 20
img_dir = os.path.join(project_dir, 'embs')
embs = [f for f in os.listdir(img_dir) if f.endswith('ch2.tif')]
i = 0
num_replicates = 10

emb = embs[i]
print(emb)

replicates = []

for i in range(num_replicates):
    img_path = os.path.join(img_dir, emb)
    hp = find_hatching.find_hatching_point(img_path)

    vnc_len = get_length_centerline(img_path, hp, interval=interval)
    replicates.append(vnc_len)

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle(f'VNC length estimation for {emb} (n=10)')

x = list(range(0, len(replicates[0])*interval, interval))
for i,l in enumerate(replicates):
    ax.plot(x, l)

plt.tight_layout()

### Effect of sampling when measuring VNC length

Different intervals were selected to check how they affect the change in the VNC length.
The idea is to also see if the measurements taken manually can be directly compared with the values in here. 

In [ ]:
img_dir = os.path.join(project_dir, 'embs')
imgs = [f for f in os.listdir(img_dir) if f.endswith('ch2.tif')]

i = 0
PIXEL_WIDTH = 1.62
img = imread(os.path.join(img_dir , imgs[i]), key=range(0,1000))

img_roi = roi.get_roi(img, window=1)
vnc_lengths = vnc_length.measure_VNC(img_roi)*PIXEL_WIDTH

reg = vnc_length.fit_regression(vnc_lengths)

windows = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
deltas = []
for w in windows:
    vnc_samples = vnc_lengths[::w]
    reg = vnc_length.fit_regression(vnc_samples)
    deltas.append((reg(vnc_samples.size-1)-reg(0))/reg(0))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle("% condensation x sampling interval")
ax.plot(windows, deltas, 'r:o');